# Calibrate model to market implied volatility

In [22]:
import numpy as np
import plotly.graph_objects as go

import sys, os


In [23]:

sys.path.append(os.path.abspath(".."))


from monte_carlo_pricer import mc_pricer
from volatility_models import build_local_vol_surface

In [24]:
strikes = np.linspace(80, 120, 30)
maturities = np.linspace(0.1, 2, 30)
S, T = np.meshgrid(strikes, maturities)
vol = 0.2 + 0.1*np.exp(-(S-100)**2/200) + 0.05*T #synthetic volatility surface



In [25]:
fig = go.Figure(data=[go.Surface(x=S, y=T, z=vol)])
fig.update_layout(title="Volatility Surface",
                  scene=dict(xaxis_title="Strike",
                             yaxis_title="Maturity",
                             zaxis_title="Vol"))
fig.show()

## A. Local Volatility Model

Vol is a function of price & time:

$$\sigma = \sigma(S,t)$$

MC simulation becomes:

$$dS = \mu S\,dt + \sigma(S,t)S\,dW$$

You need **local vol surface** (Dupire Formula), calibrated from the market.

We look at how option prices change with:

- **maturity** → $\partial C / \partial T$
- **strike** → $\partial^2 C / \partial K^2$

Those sensitivities contain information about volatility.

Dupire says:

$$\sigma^2_{\text{local}}(K,T) = \frac{\frac{\partial C}{\partial T}}{\frac{1}{2}K^2\frac{\partial^2 C}{\partial K^2}}$$




In [27]:
import yfinance as yf

ticker = yf.Ticker("NVDA")
expiry = ticker.options[0]             # one expiry for now
chain = ticker.option_chain(expiry)
spot = ticker.info['currentPrice'] 

calls = chain.calls  

# we could filter bid/ask zero contracts
# calls = calls[(calls["bid"] > 0) & (calls["ask"] > 0)]     

# we could also restrict strikes near ATM (far strikes are messy)
calls = calls[(calls["strike"] > spot)]    #strike above spot

calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
36,NVDA260102C00190000,2025-12-30 17:22:35+00:00,190.0,0.95,0.95,0.96,-0.51,-34.931507,96171,75353,0.231697,False,REGULAR,USD
37,NVDA260102C00192500,2025-12-30 17:21:37+00:00,192.5,0.36,0.36,0.37,-0.32,-46.376812,29457,86842,0.229012,False,REGULAR,USD
38,NVDA260102C00195000,2025-12-30 17:22:23+00:00,195.0,0.13,0.12,0.13,-0.15,-55.555557,26445,85168,0.233406,False,REGULAR,USD
39,NVDA260102C00197500,2025-12-30 17:22:15+00:00,197.5,0.06,0.05,0.06,-0.06,-50.000000,6712,74156,0.253914,False,REGULAR,USD
40,NVDA260102C00200000,2025-12-30 17:21:53+00:00,200.0,0.04,0.03,0.04,-0.03,-49.999996,15017,60675,0.287117,False,REGULAR,USD
41,NVDA260102C00202500,2025-12-30 17:19:27+00:00,202.5,0.02,0.02,0.03,-0.02,-50.000000,2264,13450,0.322272,False,REGULAR,USD
42,NVDA260102C00205000,2025-12-30 17:16:10+00:00,205.0,0.01,0.01,0.02,-0.02,-50.000000,1838,14799,0.347663,False,REGULAR,USD
43,NVDA260102C00207500,2025-12-30 17:10:59+00:00,207.5,0.01,0.01,0.02,-0.01,-50.000000,828,3542,0.390631,False,REGULAR,USD
44,NVDA260102C00210000,2025-12-30 16:59:37+00:00,210.0,0.01,0.00,0.01,0.00,0.000000,639,10403,0.398444,False,REGULAR,USD
45,NVDA260102C00212500,2025-12-30 15:23:20+00:00,212.5,0.01,0.00,0.01,0.00,0.000000,348,4344,0.437506,False,REGULAR,USD


In [28]:

def build_IV_surface(ticker_symbol, option_type="call", moneyness_filter=0.5):
    """
    Build raw implied volatility surface from Yahoo Finance.

    Returns:
        market_surface[expiry][strike] = IV
    """

    ticker = yf.Ticker(ticker_symbol)
    expiries = ticker.options
    spot = ticker.info.get("currentPrice", None)

    market_surface = {}

    for expiry in expiries:
        chain = ticker.option_chain(expiry)

        opt = chain.calls if option_type == "call" else chain.puts

        # we could filter out bid/ask zero contracts (non-tradable quotes)
        # opt = opt[(opt["bid"] > 0) & (opt["ask"] > 0)]

        # Optional ATM filter (keeps strikes around spot)
        if spot is not None:
            opt = opt[(opt["strike"] > spot*(1-moneyness_filter)) & 
                      (opt["strike"] < spot*(1+moneyness_filter))]

        strikes = opt["strike"].values
        IV = opt["impliedVolatility"].values

        if len(strikes) > 0:
            market_surface[expiry] = dict(zip(strikes, IV))

    return market_surface


# market_surface = build_IV_surface("NVDA")


#### some comments:
- Many strikes/maturities have no liquidity 
- Some IVs are near 0 or extremely high



In [29]:
spot

187.88

In [30]:
len(ticker.options)

20

In [31]:
# plot a market surface:
maturities = np.array(ticker.options)
market_surface = {}
for expiry in maturities:
    chain = ticker.option_chain(expiry)
    strikes = chain.calls["strike"]
    IV = chain.calls["impliedVolatility"]
    market_surface[expiry] = dict(zip(strikes, IV))
    
    

In [32]:


# strikes = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
# maturities = list(market_surface.keys())

# Z = np.zeros((len(maturities), len(strikes)))
# for i,m in enumerate(maturities):
#     for j,s in enumerate(strikes):
#         Z[i,j] = market_surface[m].get(s, np.nan)

# X, Y = np.meshgrid(strikes, maturities)

# fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
# fig.update_layout(title="Volatility Surface",
#                   scene=dict(xaxis_title="Strike",
#                              yaxis_title="Maturity",
#                              zaxis_title="Vol"))
# fig.show()


In [33]:
# #Trim to minimum common strikes across maturities
# min_len = min(len(v) for v in market_surface.values())

# strikes_common = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
# strikes_common = strikes_common[:min_len]   # first min_len 

# maturities = list(market_surface.keys())

# Z = np.zeros((len(maturities), min_len))
# for i,m in enumerate(maturities):
#     sorted_strikes = sorted(market_surface[m].keys())[:min_len]
#     for j,s in enumerate(sorted_strikes):
#         Z[i,j] = market_surface[m][s]

# X, Y = np.meshgrid(strikes_common, maturities)

# fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
# fig.update_layout(scene=dict(
#     xaxis_title="Strike",
#     yaxis_title="Maturity",
#     zaxis_title="Vol"))
# fig.show()


In [34]:
#### Comments:
# mixing all strikes without aligning them properly ( same lenght but not same set of Ks)
common_strikes = set.intersection(*(set(d.keys()) for d in market_surface.values())) #built-in to return elements common to all sets
common_strikes = sorted(common_strikes)

maturities = list(market_surface.keys())  # expiry axis

# build iv surface:
Z = np.zeros((len(maturities), len(common_strikes)))

for i, m in enumerate(maturities):
    for j, s in enumerate(common_strikes):
        Z[i, j] = market_surface[m][s]



X, Y = np.meshgrid(common_strikes, maturities)

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(
    title="Volatility Surface (Common Strikes Aligned)",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Maturity",
        zaxis_title="Implied Volatility"
    )
)
fig.show()


In [35]:
# test case since i get nan value from yahoo market surface..
K = np.array([80, 90, 100, 110, 120])
T = np.array([0.25, 0.5, 1.0, 2.0])

# Black-Scholes style smile
def smile(k): return 0.18 + 0.0005*(k-100)**2/5

IV_grid = np.array([
    [smile(k)     for k in K],       # short-dated, mild smile
    [smile(k)*1.05 for k in K],      # steeper with maturity
    [smile(k)*1.10 for k in K],
    [smile(k)*1.15 for k in K],
])

market_surface = {
    "T0.25": dict(zip(K, IV_grid[0])),
    "T0.50": dict(zip(K, IV_grid[1])),
    "T1.00": dict(zip(K, IV_grid[2])),
    "T2.00": dict(zip(K, IV_grid[3])),
}



In [36]:
KK, TT = np.meshgrid(K, T)

fig = go.Figure(data=[go.Surface(
    x=KK,   # Strike axis
    y=TT,   # Maturity axis
    z=IV_grid, # Vol matrix
    colorbar=dict(title="Vol")
)])

fig.update_layout(
    title="Implied Volatility Surface",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Maturity",
        zaxis_title="IV"
    )
)
fig.show()


In [37]:


LV = build_local_vol_surface(100, market_surface, r=0.03)

print("Local Vol at ATM:", LV(1.0, 100)[0,0])


IV_grid:
 [[0.22   0.19   0.18   0.19   0.22  ]
 [0.231  0.1995 0.189  0.1995 0.231 ]
 [0.242  0.209  0.198  0.209  0.242 ]
 [0.253  0.2185 0.207  0.2185 0.253 ]]
0.21999999999999992
0.1899999999999999
0.17999999999999988
0.18999999999999992
0.2199999999999999
0.23100000000000004
0.19949999999999998
0.18899999999999997
0.19949999999999998
0.231
0.2420000000000001
0.20899999999999996
0.19799999999999998
0.20900000000000005
0.24200000000000005
0.2530000000000001
0.21849999999999997
0.207
0.21849999999999997
0.253
Local Vol at ATM: 0.3176966451246616


#### Local Vol MC Price: 

In [40]:

def european_call_payoff(path, K):
    return np.maximum(path[-1] - K, 0.0)



price, ci, stderr, beta = mc_pricer(
    payoff_fn=european_call_payoff,
    payoff_args=[K],
    S0=100, r=0.03, sigma=None, # sigma ignored!
    T=1.0, N=250, M=20000,
    sim_method="local_vol",  
    LV=LV,   
    seed=42
)


In [39]:
print("Local Vol MC Price =", price)
print("95% CI =", ci)
print("std error =", stderr)

Local Vol MC Price = 12.2159481233037
95% CI = (11.662425271528742, 12.76947097507866)
std error = 0.28241480769089455


# Stochastic volatility models 

## B. Heston stochastic volatility model

Vol follows its own random process:

$$dS = \mu S\,dt + \sqrt{v_t}S\,dW_1$$
$$dv_t = \kappa(\theta - v_t)\,dt + \sigma_v\sqrt{v_t}\,dW_2$$

- Two correlated random numbers per step
- Parameters calibrated to implied vol surface/market surface


## C. SABR model

Used a lot in interest rates:

$$dF = \sigma_t F^\beta\,dW_1$$
$$d\sigma = \alpha\sigma\,dW_2$$

Again, volatility is random → simulate both


Stress test prices under market shocks